# Diffusion Monte Carlo Method

Description of theory and stuff

A diffusion monte carlo simulation for the 3-dimensional harmonic oscillator, ground state energy and wave function 

$$E_0 = \frac{2}{3}, \hspace{2em} \psi_0 = \frac{e^{-r^2/2}}{(2\pi)^{3/2}}$$

using $m=\omega=\hbar=1$

In [1]:
import numpy as np
import numpy.random as random

dim = 3

Potential of the harmonic oscillator in three dimensions

$$V(\mathbf x) = \frac{1}{2}\mathbf x ^2$$

In [2]:
def pot(x):
    return 0.5 * x.dot(x)

Set the time step size, the target number of walkers as well as the target number of time steps

In [95]:
dt = 0.05
N_T = 300
T_steps = 4000
alpha = 0.1

The first 20% of the time steps are used as thermalization steps, after that, the measured values are reset and the real simulation continues

In a timestep
1. Compute one DMC step on each walker
2. Remove dead walkers
3. Adjust $E_T$ to drive $N$ towards $N_T$
4. Accumulate data to measure $\left<E\right>$, its variance and the ground state wave function

In [97]:
# Initialize walkers
r = np.zeros((N_T, 3))
N = N_T
E_T = 0 # initial guess for the ground state energy

thermal_steps = int(0.2*T_steps)

# Time step
for i in range(T_steps):
    N_0 = N
    
    r_new = np.zeros((0,3))
    
    # Monte Carlo Step
    
    # Diffusion step
    r += random.normal(size=(N_0,3)) * np.sqrt(dt)
    
    q = np.apply_along_axis(lambda x: np.exp(-dt * (pot(x) - E_T)), 1, r)
    
    for j in range(N_0):
        if q[j] - int(q[j]) > random.uniform():
            count_new = int(q[j]) + 1
        else:
            count_new = int(q[j])
        for c in range(count_new):
            r_new = np.append(r_new,r[j:j+1,:],axis=0)

    N = r_new.shape[0]
    r = r_new
    
    E_T += alpha * np.log(N_T / float(N))